In [ ]:
import json
import random
import nltk
import tensorflow
import pickle

import numpy as np
#from numpy import arange
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.models import load_model


In [ ]:
intents = json.loads(open("intents removed.json").read())
lemmatizer = WordNetLemmatizer()

words = []
classes = []
doc_x = []
doc_y = []
ignore = ['?','.',',','!',')','(',':',';','\'']

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_tokens = nltk.word_tokenize(pattern)
        words.extend(word_tokens)
        doc_x.append(word_tokens)
        doc_y.append(intent["tag"])
    
    if intent["tag"] not in classes:
        classes.append(intent["tag"])
        
print("words:", words)
print("\n\n\n\n\n\doc_x:", doc_x )

In [ ]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore]
words = sorted(set(words))
classes = sorted(classes)

pickle.dump(words, open("word_CR3.pkl","wb"))
pickle.dump(classes, open("classes_CR3.pkl","wb"))

In [ ]:
training =[]
output = []
output_empty = [0]*len(classes)

for i, doc in enumerate(doc_x):
    bag = []
    doc_words = doc
    doc_words = [lemmatizer.lemmatize(wrd.lower()) for wrd in doc_words]
    #print(doc_words)
    for w in words:
        if w in doc_words:
            bag.append(1)
        else:
            bag.append(0)
    
    output_row = output_empty[:]
    output_row[classes.index(doc_y[i])] = 1
    
    training.append(bag)
    output.append(output_row)

#print("\n training: ",training)
#print("\n\n\n\n output: ", output)
training = np.array(training)
output = np.array(output)

#print(words)
#print(doc_words)
print(training)
print(output)

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(training[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(output[0]), activation = 'softmax'))

sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss='categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
hist_CR3 = model.fit(training, output, epochs = 200, batch_size = 5, verbose=1)
model.save("Chatbot_CR3.h5",hist_CR3)
print("MODEL CREATED")

In [ ]:
words = pickle.load(open("word_CR3.pkl","rb"))
classes = pickle.load(open("classes_CR3.pkl","rb"))

model = load_model("Chatbot_CR3.h5")

In [ ]:
def break_sentence(sentence):
    sen_tokens = nltk.word_tokenize(sentence)
    sen_tokens = [lemmatizer.lemmatize(w.lower()) for w in sen_tokens]
    return sen_tokens

In [ ]:
def bag_of_words(sentence):
    word = break_sentence(sentence)
    word_bag = [0]*len(words)
    for wrd in word:
        for i,w in enumerate(words):
            if w == wrd:
                word_bag[i]=1
    return np.array(word_bag)

In [ ]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    
    #error_threshold = 0.25
    result = [[i,r] for i,r in enumerate(res)]# if r > error_threshold]
    #print(result)
    result.sort(key = lambda x:x[1], reverse = True)
    #print(result)
    #result_categories = []
    #for r in result:
        #result_categories.append({'intents':classes[r[0]], 'probability': r[1]})
    return result

In [ ]:
def get_response(probable_intents, intents_json):
    tag = classes[probable_intents[0][0]]
    probability = probable_intents[0][1]
    print(probability)
    print(tag)
    total_intents = intents_json['intents']
    results = ["I didn't get that.", "Please check the spellings or try asking something else.","I am sorry, I didn't quite get that.","Please ask again, I'll try to answer better this time.","This is outside my Knowledge Please try asking something else :("]
    if (probability < 0.90): #((tag == "yes") and (probability in arange(0.50,0.95,0.01))) or 
        ##result = "I didn't get that, please check the spellings or try asking something else."
        res = random.choice(results)
        return res
    else:
        for i in total_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
        return result    

In [ ]:
#!pip install transformers
from transformers import pipeline

def sentiment_analysis(text):
    sentiment = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa')
    emotion_label  = sentiment(text)
    emotion = emotion_label[0]['label']
    print(emotion)
    return emotion
    

In [ ]:
import requests
def song_recommendation(emotion):
    #emotion = input("Enter your feeling ")
    url = "http://ws.audioscrobbler.com/2.0/?method=tag.gettoptracks&tag={0}&api_key=19e1d5326b6fe057e0e031b6c1131136&format=json&limit=10".format(emotion)
    response = requests.get(url)
    song_data = response.json()
    song_dict = {}
    for i in range(5):
        r = song_data['tracks']['track'][i]
        song_dict[r['name']] = r['url']
    print("Song recommendations:")
    for n,u in song_dict.items():
        print("Song name:", n)
        print("Song url:", u)

In [ ]:
print("Chatbot is ready!! Start chatting and (Type 'quit' to stop chatting and get the song recommendations). Please refrain from using short forms and try writing full sentences. \nEg: great  (wrong) \n  : I'm great  (correct)    ")
print("Chatbot: Hey, there!")
text =""
while True:
    msg = input("User: ")
    if msg.lower() == "quit":
        break
    ints = predict_class(msg)
    res = get_response(ints, intents)
    print("Chatbot:",res)
    text = text+" "+msg
print(text)
senti = sentiment_analysis(text)
song_recommendation(senti)
    